# Exercise 2 - Convolutional Neural Nets (CNN's)

In the last notebook you built a feedforward net, and found that it could recognise handwritten digits remarkably well. However, when trying to recognise diverse objects on images we ran into performance issues. In this notebook you will learn to implement a simple CNN in Keras and apply it to CIFAR-10.

The set-up of this notebook is similar to the previous. You are expected to complete certain parts of the code to be able to run it.

## 1 - Packages

First we begin by importing the required packages

In [1]:
#download required datasets for this notebook (might take a bit, be patient!)
from urllib.request import urlretrieve
import os
from zipfile import ZipFile

def download(url, file):
    if os.path.isfile(file):
        os.remove(file)
    if not os.path.isfile(file):
        print("Download file... " + file + " ...")
        urlretrieve(url,file)
        print("File downloaded")
        
def unzip(file):
    with ZipFile(file) as f:
        f.extractall()
    print('unzipped file: %s\n' %file)

try:
    download('http://www.paulvangent.com/files/DL_Course/utils_day1.py', 'utils_day1.py')
except:
    download('https://onedrive.live.com/download?cid=39383A5AFCD95065&resid=39383A5AFCD95065%21754588&authkey=ANqFqQOFr4Iqy1A', 'utils_day1.py')

import numpy as np #import numpy package
import matplotlib.pyplot as plt #import plotting library
import utils_day1 as utils #import helper functions

#import required keras modules
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Flatten, Conv2D, Conv1D, MaxPooling2D, RepeatVector

Download file... utils_day1.py ...
File downloaded


Using TensorFlow backend.


## 2 - Dataset

We load the CIFAR-10 dataset again. 

**Note** that we do not need to flatten the images. Remember from the lecture that a convolution can operate on two dimensional image data.

In [0]:
from keras.datasets import cifar10

(X_train, Y_train), (X_val, Y_val) = cifar10.load_data()

We do, however, still need to apply one-hot encoding and normalise the images. In the last notebook you did this with [numpy.eye()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.eye.html).

Your task here is to use a built-in keras function, called [to_categorical](https://keras.io/utils/#to_categorical). Complete the code below to convert Y_train and Y_val to one-hot encoding using this function. Look at the link for a hint, and note that the dataset has **10 classes**.

In [0]:
from keras.utils import to_categorical

#Apply one-hot encoding
##Start your code here (approx. 2 lines)
Y_train = None
Y_val = None
##End of your code

#Normalise the images
##Start your code here (approx. 2 lines)
X_train = None
X_val = None
#End your code here

print('Shape of first training label: ' + str(Y_train[0].shape))
print('Shape of last validation label: ' + str(Y_val[-1].shape))
print('mean pixel value of first training example: ' + str(np.mean(X_train[0])))
print('mean pixel value of last validation example: ' + str(np.mean(X_val[-1])))

***

**Expected output:**

Shape of first training label: (10,)  
Shape of last validation label: (10,)  
mean pixel value of first training example: 0.405675551471  
mean pixel value of last validation example: 0.423383884804

## 3 - Defining a CNN in Keras

Implementing a CNN in Keras is very similar to implementing a feedforward network, with the exception that we use different layer types. For an overview of all available convolutional layers, see [this link](https://keras.io/layers/convolutional/).

[**Please take a look here for some additional reading material regarding Convolutions**](https://arxiv.org/pdf/1603.07285v1.pdf)


***

Your task is to implement a CNN with the following structure:

- Convolutional 2D input layer. Use the following parameters:
    * 32 filters
    * kernel size of (3, 3)
    * input shape equal to input_shape
    * relu activation
- Max pooling 2D layer. Use a poolsize of (2, 2)
- Regularisation layer. Use a dropout rate of 20%
- Flattening layer 
- Dense layer, use 256 units and relu activation.
- Output layer with 10 classes and softmax activation.


Can you explain **why** we need to add a flatten layer between the CNN part of the network and the Dense layers?

In [0]:
def baseline_model():
    model = Sequential()
    ##Add your code here (approx. 6 lines)

    
    
    
    
    
    ##End of your code
    
    
    return model

We then initialise the model:

In [0]:
model = baseline_model()

print(model.summary())

***
**Expected Output:**

![model summary](http://www.paulvangent.com/files/DL_Course/day1_images/cnn_network.jpg)

**Compare this to the FeedForward network summary from the previous notebook (we've recreated the image below).**

![FFN model](http://www.paulvangent.com/files/DL_Course/day1_images/feedforwardmodel.jpg)

**Can you explain why the CNN has fewer parameters, yet gets much better results?**


***
## 4 - Loss Functions and Optimization

This section will detail the loss function and optimization of your network.

### 4.1 Loss Function
Remember from the lecture this morning, that in order to optimize the model we need to express its accuracy. Expressing the mistakes the model makes is done with a loss function, which quantifies the distance between the expected output and the predicted output. An easy example to understand is that of the logistic loss function:

![logloss](http://www.paulvangent.com/files/DL_Course/day1_images/logloss.jpg)

**Note:** **y** is often used for the true label, and y-hat (**ŷ**) for the predicted value.

To see what it does, note that the function has two sides (left and right of ‘+’). The function expresses the similarity of prediction and true value. It 'punishes' larger deviations more severly. When the prediction and true value are very similar the loss is low, when they differ a lot the loss is high. To think visually, consider the plot below:

![loglossgraph](http://www.paulvangent.com/files/DL_Course/day1_images/log_loss_graph.png)

Still not 100% clear on what happens? Consider two opposite cases:

***
<center>**y=1**  
The right side of the formula falls off because (1 – y) = 0, so that implies that for -log(ŷ) you want ŷ to be 1 (or close to 1), since log(1)=0.  </center>
  
<center>**y=0**  
Now the left side of the formula falls off (0 log (ŷ) = ..). Now we need log(1 – ŷ) to be close to 0 to mach y, which happens if ŷ=0.</center>

***

Logloss is only applicable when there are two possible outcomes (binary classification). In the exercise below we will be using **categorical crossentropy**, which is extremely similar to logloss, except that it can handle more than two possible outcomes.

[**You can see the mathematical definition of the categorical crossentropy here**](https://rdipietro.github.io/friendly-intro-to-cross-entropy-loss/)

***

### 4.2 - Optimizer
The loss function allows us to quantify the mistakes made by model, which makes it possible to reduce them. Remember from the lectures that the optimizer will try to gradually step towards a point of minimum loss. Consider a hypothetical, one-dimensional loss function to illustrate:

![onedimloss](http://www.paulvangent.com/files/DL_Course/day1_images/gradientdescent0.jpg)

Here we want to minimise **C(W)**, because a low loss means the difference between predictions and actual values is small. In the context of the figure: the optimizer decides which is 'downhill', and updates the model's weights so that the loss decreases:

![stepping to minimum](http://www.paulvangent.com/files/DL_Course/day1_images/gradientdescent2.jpg)

An excellent overview of the different optimizers [can be found here](http://ruder.io/optimizing-gradient-descent/).

In this notebook we use the Adam optimiser. In many cases Adam will converge quickly, which is useful here because many of you are running the code without a GPU.

***
**Exercise:**  
We still need to compile the model, meaning we will be asking Keras to transform our model instructions into machine-readable code. Compile the model with a 'categorical_crossentropy' loss and an 'adam' optimizer. 

Hint: refer to how we did this in the previous notebook if you're unsure how to do this.

In [0]:
##Add your code here (approx. 1 line)

##End of your code

Your task is now to fit the model for 10 epochs with a batch size of 256. Remember to also set the validation data sets!

Hint: Refer to the previous notebook on how to do so if you are unsure.

In [0]:
##Add your code here (Approx. 1 line)
history = None
##End of your code.


***
Note that we added 'history = ' before the model.fit command. Remember that by default, a Keras model will return a history object containing information on the training epochs. Run the cell below to plot the results of your training phase

In [0]:
utils.plotter(history)

Take a look at the plot. **What do you think? Is the model overfitting or not? Why (not)?** 

Based on this graph, would you try to run the model for more epochs or not?

***

## 5 - Expanding the CNN (optional)

In this last section of the notebook we will be expanding the CNN network. A bigger network will have more capacity to fit the variance in the dataset.

This section is optional because without a discrete GPU (which most of your laptops will not have), training will take long. Once it works you can turn on the model at night and let the laptop run for a while.

We have copied an example of the baseline model below. Your task is to add two Conv2D layers to it.

- Convolutional 2D input layer. Use the following parameters:
    * 64 filters
    * filtersize of (5, 5)
    * use 'same' padding
    * input shape of (32, 32, 3)
    * relu activation
- Max Pooling layer with a poolsize of 2
- Convolutional 2D input layer. Use the following parameters:
    * 128 filters
    * filtersize of (3, 3)
    * use 'same' padding
    * input shape of (32, 32, 3)
    * relu activation

In [0]:
def expanded_model(input_shape=(32, 32, 3), classes=10):
    model = Sequential()
    ##Add your code here (approx. 4 lines)

    
    
    
    
    ##End of your code
    model.add(Dropout(rate=0.2))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(rate=0.2))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(classes, activation='softmax'))
    ##End of your code
    
    
    return model

In [0]:
model = expanded_model()

model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

Your task here is to again fit the model. Use the same setting as earlier (10 epochs, batchsize 256).

In [0]:
##Add your code here
history = None
##End of your code.

utils.plotter(history) #plot results

So, how did the model perform? Our iteration reached ~73% in the end. **Would you try extra epochs? Why (not)?**

You might wonder, how does the model work on the MNIST dataset from the first notebook? It would take too much time to let the model converge on your laptop in the few hours we have today. We ran it for 25 epochs on a gpu though, and it reached **98.94%** validation accuracy. That's pretty good!

## 6 - Wrapping up

You've reached the end of the second notebook. Hoo-ray, you're now a certified Deep Learner! You might now want to go online, get some bigger datasets, and try larger networks for yourself. You might also want to try applying what you learned to your own data.

Be warned that beyond this point a GPU becomes mandatory. Even a low-budget one can speed up training dramatically, making the difference between fitting a model for a night or over a month.

![GPU MANDATORY](http://www.paulvangent.com/files/DL_Course/day1_images/needgpu.jpg)

***

There is one last optional notebook available. This will deal with more advanced Keras features, such as saving your model for re-use later.

[Go to it now!](Exercise 3 - Advanced Keras functionality.ipynb)